In [1]:
import pandas as pd
df = pd.read_csv('HR_Data_Predict Employee Turnover.csv')
col_nums = df.columns.tolist()
print('column names:')
print(col_nums)
print('\nSample data')
df.head()

column names:
['satisfaction_level', 'last_evaluation', 'number_project', 'average_montly_hours', 'time_spend_company', 'Work_accident', 'left', 'promotion_last_5years', 'Department', 'salary']

Sample data


,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,Department,salary
0,0.38,0.53,4,157,3,0,1,0,sales,medium
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [2]:
df.dtypes

,0
satisfaction_level,float64
last_evaluation,float64
number_project,int64
average_montly_hours,int64
time_spend_company,int64
Work_accident,int64
left,int64
promotion_last_5years,int64
Department,object
salary,object


In [3]:
df.isnull().any()

,0
satisfaction_level,False
last_evaluation,False
number_project,False
average_montly_hours,False
time_spend_company,False
Work_accident,False
left,False
promotion_last_5years,False
Department,False
salary,False


In [4]:
df.shape

(14999, 10)

In [5]:
df['Department'].unique()

array(['sales', 'accounting', 'hr', 'technical', 'support', 'management',
       'IT', 'product_mng', 'marketing', 'RandD'], dtype=object)

In [6]:
import numpy as np
df['Department']=np.where(df['Department']=='support',1,df['Department'])
df['Department']=np.where(df['Department']=='technical',2,df['Department'])


In [7]:
df['left'].value_counts()

,count
left,
0,11428
1,3571


In [9]:
cat_vars=['Department','salary']
for var in cat_vars:
    cat_list='var'+'_'+var
    cat_list = pd.get_dummies(df[var], prefix=var)
    df1=df.join(cat_list)
    df=df1

In [10]:
df.drop(df.columns[[8, 9]], axis=1, inplace=True)
df.columns.values

array(['satisfaction_level', 'last_evaluation', 'number_project',
       'average_montly_hours', 'time_spend_company', 'Work_accident',
       'left', 'promotion_last_5years', 'Department_1', 'Department_2',
       'Department_IT', 'Department_RandD', 'Department_accounting',
       'Department_hr', 'Department_management', 'Department_marketing',
       'Department_product_mng', 'Department_sales', 'salary_high',
       'salary_low', 'salary_medium'], dtype=object)

In [12]:
df_vars = df.columns.values.tolist()
y = ['left']
X = [i for i in df_vars if i not in y]

In [19]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
# Specify n_features_to_select explicitly as a keyword argument
rfe = RFE(model, n_features_to_select=10)
rfe = rfe.fit(df[X], df[y])
print(rfe.support_)
print(rfe.ranking_)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel

[ True  True False False  True  True  True False False False  True False
  True  True False False False  True  True False]
[ 1  1  3 11  1  1  1  5  4  9  1  6  1  1  7 10  8  1  1  2]


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [22]:
cols = ['satisfaction_level' , 'last_evaluation' ,
        'time_spend_company' ,  'Work_accident' , # Corrected column name to 'Work_accident' if it exists
        'Department_accounting' , 'Department_hr', # Corrected typo from 'Deaprtment_accounting' to 'Department_accounting'
        'salary_high' , 'salary_low']
X = df[cols]
y = df['left']

In [33]:
from sklearn.model_selection import train_test_split # Import train_test_split from the correct module
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [34]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

LogisticRegression()

In [35]:
from sklearn.metrics import accuracy_score
print('Logistic Regression Accuracy:{:.3f}'.format(accuracy_score(y_test,logreg.predict(X_test))))

Logistic Regression Accuracy:0.770


In [36]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train, y_train)

RandomForestClassifier()

In [37]:
print('Random forest Accuracy {:.3f}'.format(accuracy_score(y_test,rf.predict(X_test))))

Random forest Accuracy 0.978
